In [24]:
import pandas as pd

data = pd.read_csv('1987_coded.csv')
data.drop(['same.pl2', 'same.mn2', 'same.vc2', 'vc2', 'same.ant2', 'same.ns2', 'size', 'syl', 'sylnum',	'monosyl'], axis=1, inplace=True)
data.columns

# count frequency 
# prune columns
# empty rows
prep 

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (7,8,12,16,17,21,22,31,32,33,34,38,44) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Index(['prep', 'stem_stress', 'stem', 'epen', 'prep_c', 'c1', 'c2', 'c3', 'c4',
       'stem2', 'c1_2', 'c2_2', 'c3_2', 'prep_pl', 'pl1', 'pl2', 'pl3', 'pl4',
       'prep_mn', 'mn1', 'mn2', 'mn3', 'mn4', 'prep_vc', 'vc1', 'prep_ns',
       'ns1', 'ns2', 'ns3', 'ns4', 'prep_ant', 'ant1', 'ant2', 'ant3', 'ant4',
       'same.pl1', 'same.mn1', 'same.vc1', 'same.ant1', 'same.ns1'],
      dtype='object')

prep 
stem
epen
prep_c
stem2
pl1
mn1
vc1
ant1
same.mn1
same.ant1

In [2]:
left_df = pd.DataFrame(data.input.unique(), columns=['input'])

# 1. Copy input column to create output column
# https://stackoverflow.com/questions/32675861/copy-all-values-in-a-column-to-a-new-column-in-a-pandas-dataframe/32682095
left_df['output'] = left_df['input']

# ref : https://www.codeforests.com/2020/09/27/pandas-split-one-row-of-data-into-multiple-rows/
# 2. Duplicate each row and reset index
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html
left_df = left_df.loc[left_df.index.repeat(2),:].reset_index(drop=True)

# 3. Get duplicated row indexes
idx_duplicate = left_df.duplicated(keep="first")

# 4. Replace output value of duplicated rows from s_ to so_
left_df.loc[idx_duplicate, 'output'] = left_df.loc[idx_duplicate, 'output'].replace(regex=r's_', value='so_')
left_df

,input,output
0,s_бдениями,s_бдениями
1,s_бдениями,so_бдениями
2,s_бдительностью,s_бдительностью
3,s_бдительностью,so_бдительностью
4,s_бзиками,s_бзиками
...,...,...
21101,s_штырями,so_штырями
21102,s_шхуной,s_шхуной
21103,s_шхуной,so_шхуной
21104,s_шхуны,s_шхуны


In [3]:
right_df = data.drop(['input'], axis=1)
right_df

,output,freq,dep,ccc,ocp_sib,ocp_ant
0,s_бдениями,1,0,1,0,0
1,s_бдительностью,2,0,1,0,0
2,s_бзиками,1,0,1,1,1
3,s_блаватником,1,0,1,0,0
4,s_благами,1,0,1,0,0
...,...,...,...,...,...,...
10847,so_штырьком,1,1,0,2,0
10848,so_штыря,1,1,0,2,0
10849,so_штырями,2,1,0,2,0
10850,so_шхуной,1,1,0,2,0


In [15]:
new_df = left_df.join(right_df.set_index('output'), how='left', on='output')
new_df['freq'].fillna(0, inplace=True)
new_df['dep'].fillna(1, inplace=True)
new_df['ccc'].fillna(0, inplace=True)
new_df['ocp_sib'].fillna(0, inplace=True)
new_df['ocp_ant'].fillna(0, inplace=True)
new_df.astype({'freq' : int, 'dep' : int, 'ccc' : int, 'ocp_sib' : int, 'ocp_ant' : int})

,input,output,freq,dep,ccc,ocp_sib,ocp_ant
0,s_бдениями,s_бдениями,1,0,1,0,0
1,s_бдениями,so_бдениями,0,1,0,0,0
2,s_бдительностью,s_бдительностью,2,0,1,0,0
3,s_бдительностью,so_бдительностью,0,1,0,0,0
4,s_бзиками,s_бзиками,1,0,1,1,1
...,...,...,...,...,...,...,...
21101,s_штырями,so_штырями,2,1,0,2,0
21102,s_шхуной,s_шхуной,0,1,0,0,0
21103,s_шхуной,so_шхуной,1,1,0,2,0
21104,s_шхуны,s_шхуны,0,1,0,0,0


In [11]:
new_df.to_csv('mxnt_output.csv', index=False)